### ネット上に氾濫する自己組織化マップ（SOM）の情報を俺が整理する
https://qiita.com/ae14watanabe/items/7c70924798c8125c05eb

### とある科学の備忘録
#### 【Python】自己組織化マップ（SOM）の説明とPythonコード
https://shizenkarasuzon.hatenablog.com/entry/2019/02/03/174551

#### SOMとは
- 自己組織化マップ（SOP、Self-organizing maps） はフィンランドの研究者、T. Kohonenが発明したニューラルネットの一種です
- SOMは教師なし学習を行い、高次元のデータセットを低次元空間（主に二次元）に写像するのに使用されます
- 2次元に写像した場合、データ分布が地図（Topographic Map）のように可視化されるため、この地図をデータマイニングに用いられることがよくあります

#### 紹介するサンプル
- SOMは多次元のデータを可視化するのに適しています
  - 「色」という三次元データを分類・可視化してみたいと思います
- パソコンの世界では、「色」はRed、Blue、Greenの三色の混ぜ方で決まっています
  - それぞれ0～255の強さがあります
  - 例えば、（Red, Green, Blue)= (255,0,0)は赤色、(Red, Green, Blue) = (255,255,0)は赤と緑の光の混ぜ合わせなので黄色となります
  - つまり、色には三次元のデータあるのです

- この三次元のデータをSOMに組み込み、似た色のデータを近くに配置することで、色を分類することができるのではないか？というのがSOMのアルゴリズムです

#### アルゴリズム
- 最初はノードにランダムな色を記憶させておきます
  - これは、隣同士のノードの関係がよくわからない、つまり学習が行われていない初期段階です
- まず、SOMには、大量の色データが一つずつ順番に与えられます
- それら一つ一つのデータに対して学習していくのですが、その学習アルゴリズムは以下の2STEPです
  1. 入力された色データに最も近い色を表しているノードを探す
  2. そのノードと、その周辺ノードを、入力された色データの色に近づける
    - 例えば、入力データが赤色だった場合、赤色のノードとその周りがさらに赤っぽくなります
- この2STEPを繰り返すことで、だんだんとノードがきれいになっていきます
- そして、最終的には、色が二次元マップ上にきれいに分類されます

#### サンプルプログラム

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Nx = 30
Ny = 30
learntime = 10000
alpha = 0.08
weight = np.random.random([Nx,Ny,3])

def som(ColorVec):
  min_index = np.argmin(((weight-ColorVec)**2).sum(axis=2))
  mini = int(min_index / Ny)
  minj = int(min_index % Ny)
  for i in range(-2,3):
    for j in range(-2,3):
      try:
        weight[mini+i,minj+j] += alpha * (ColorVec - weight[mini+i,minj+j])
      except:
        pass

'''
print("initshow")
im1 = plt.imshow(weight,interpolation='none')
plt.show()
'''

for time in range(learntime):
  ColorVec = np.random.rand(3)
  som(ColorVec)
  if time%500 == 0:
    print("transition@",time)
    im0 = plt.imshow(weight,interpolation='none')
    plt.show()

print("result")
im2 = plt.imshow(weight,interpolation='none')
plt.show()

### CNNの勉強(Ⅱ)-自己組織化マップの理解とPythonによるプログラム例
https://agirobots.com/blog-ai-ml-theory-self-organizing-map/

In [8]:
import numpy as np
import matplotlib.pyplot as plt
#ガウス分布の確率密度関数をインポート
from scipy.stats import norm

In [9]:
# 入力データを定義
X = np.array([[1, 0, -1, 0],
              [0, 1, 0, -1]])

In [10]:
# 出力層の全てのニューロンのマップ座標を定める
Vec = np.array([[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                [0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4]])

In [11]:
# 個々のニューロンの重みを正規分布に従ってランダムに生成
W = np.random.rand(2, 25)

# 自己組織化マップでは入力ベクトルと重みベクトルがどれだけ似ているかを、競争して学習するため内積を使用する
# それぞれが単位ベクトルになるように正規化
W = W / np.linalg.norm(W, axis=0)

In [ ]:
# 初期状態で、マップ上のニューロンがどの向きのベクトルを持っているのか、プロット
import sys
plt.figure()
plt.quiver(Vec[0, :], Vec[1, :], W[0, :], W[1, :], angles='xy', scale_units='xy', scale=1)
plt.xlim([-1,5])
plt.ylim([-1,5])
plt.grid()
plt.axes().set_aspect('equal')
plt.show()

In [14]:
#学習率cを0.01にセット
c = 0.01

#4つの入力データを10000回学習させる
for _ in range(10000):
  #iは4つの学習データから1つを選ぶ
  for i in range(4):
    #内積(入力ベクトルと重みベクトルの向きの近さ)を計算した結果、
    #最も値の大きかったニューロン(勝者ニューロン)のインデックスを取得する
    j = np.argmax(np.dot(W.T, X[:, i]),axis=0)

    #勝者ニューロンから見た時の周囲のニューロンの位置ベクトルを計算する
    D_Vec = Vec - Vec[:, j][:, np.newaxis]

    #D_Vecの列方向において、ユークリッド距離を計算する
    D = np.linalg.norm(D_Vec, axis=0)

    #勝者ニューロンを中心にガウス分布に従う割合で、入力データに
    #重みベクトルを近づけるための係数ベクトルHを取得する
    H = norm.pdf(D, scale=2)

    #全てのニューロンの重みベクトルを更新する
    W = W + c * H * (X[:, i][:, np.newaxis] - W)

    #それぞれの重みベクトルを単位ベクトルに正規化する
    W = W / np.linalg.norm(W, axis=0)

In [ ]:
# どのように重みベクトルが変化したか可視化
import sys
plt.figure()
plt.quiver(Vec[0, :], Vec[1, :], W[0, :], W[1, :], angles='xy', scale_units='xy', scale=1)
plt.xlim([-1,5])
plt.ylim([-1,5])
plt.grid()
plt.axes().set_aspect('equal')
plt.show()

### Pythonで自己組織化マップのプログラムを実装する方法【初心者向け】
https://techacademy.jp/magazine/21498

In [ ]:
!pip install somoclu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import somoclu
#%matplotlib inline

In [ ]:
c1 = np.random.rand(50, 3)/5
c2 = (0.6, 0.1, 0.05) + np.random.rand(50, 3)/5
c3 = (0.4, 0.1, 0.7) + np.random.rand(50, 3)/5
data = np.float32(np.concatenate((c1, c2, c3)))
colors = ["red"] * 50
colors.extend(["green"] * 50)
colors.extend(["blue"] * 50)
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c=colors)
labels = range(150)

In [ ]:
n_rows, n_columns = 100, 160
som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False)
som.train(data)
som.view_component_planes()

### 【python】SOMのライブラリSomocluはかなりおすすめ
https://www.haya-programming.com/entry/2018/04/07/161249#%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%9F

In [24]:
# coding: UTF-8
import numpy as np

from somoclu import Somoclu
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

In [ ]:
def main():
  # データを読み込む
  dataset = load_iris()
  X = dataset.data
  y = dataset.target

  # SOMに入れる前にPCAして計算コスト削減を測る（iris程度では無駄） 
  pca = PCA(n_components=0.95) 
  X = pca.fit_transform(X)

  # SOMの定義
  n_rows = 16
  n_cols = 24
  som = Somoclu(n_rows=n_rows, n_columns=n_cols, initialization="pca", verbose=2)

  # 学習
  som.train(data=X, epochs=1000)

  # U-matrixをファイル出力
  som.view_umatrix(labels=y, bestmatches=True, filename="umatrix.png")

if __name__ == "__main__":
  main()

In [27]:
# coding: UTF-8
import numpy as np

from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from somoclu import Somoclu
import matplotlib.pyplot as plt

In [ ]:
def main():
  print("loading data")
  digits = load_digits()
  pca = PCA(n_components=0.95)
  pca_data = pca.fit_transform(digits.data)

  # tsneで可視化
  tsne = TSNE()
  X = tsne.fit_transform(pca_data)
  fig, ax = plt.subplots()
  plt.scatter(X[:,0], X[:,1], c=digits.target/10)
    
  i = 0
  for xy, l in zip(X, digits.target):
    if i%8 == 0:    # 描画されるtextが多いと汚いので省く
      ax.annotate(l, xy=xy)
    i += 1
  plt.savefig("tsne_digits.png")

  # somで可視化
  # データを適当に省く
  sample_index = np.random.choice(X.shape[0], 400, replace=False)
  sample_X = pca_data[sample_index]
  sample_y = digits.target[sample_index]

  # som
  som = Somoclu(n_rows=30, n_columns=40, initialization="pca")
  som.train(data=sample_X, epochs=1000)
  som.view_umatrix(labels=sample_y, bestmatches=True, filename="som_digits.png")

if __name__ == "__main__":
  main()

### 自己組織化マップ（SOM: Self-Organizing Map）試作記
http://y-okamoto-psy1949.la.coocan.jp/Python/misc/SOM/

In [6]:
# 描画モジュールdrawmap.py

import matplotlib.pyplot as plt
import numpy as np

def draw_som(case_labels, xi, m):
  JM = len(xi)
  n = len(xi[0])
  g_size = len(m)

  #
  #   参照ベクトルの傾き
  #
  slopes = np.empty((g_size, g_size))
  for i in range(0, g_size):
    for j in range(0, g_size):
      count = 0
      v = 0.0
      for ix in range(-1, 2):
        for jy in range(-1, 2):
          try:
            if ix != 0 or jy != 0:
              if (i+ix) >= 0 and (j+jy) >= 0:
                v += np.absolute(m[i+ix][j+jy] - m[i][j]).sum()
                count += 1
          except:
            pass
      slopes[i][j] = v / count
  slope_max = np.ma.max(slopes[0:g_size, 0:g_size])
  slope_min = np.ma.min(slopes[0:g_size, 0:g_size])

  # 日本語表記の設定
  plt.rcParams['font.family'] = ['Yu Gothic', 'Noto Sans CJK JP', 'Hiragino Maru Gothic Pro']

  #
  #   傾きに応じた色付け
  #
  plt.figure(figsize = (7, 7))
  plt.axis('off')
  plt.title('自己組織化マップ\nSOM', fontsize = 20)
  for i in range(0, g_size):
    for j in range(0, g_size):
      vc = (slopes[i][j] - slope_min) / (slope_max - slope_min)
      plt.fill_between([i-0.5, i+0.5], [j-0.5, j-0.5], [j+0.5, j+0.5], color = (vc, 1.0-vc, 0.0), alpha = 0.5)

  m_c = []
  diffs = np.empty((g_size, g_size))
  plt.xticks([-1, g_size+1], [' ', ' '])
  plt.yticks([-1, g_size+1], [' ', ' '])
  for IN in range(JM):
    #
    #   勝者ベクトル
    #
    for i in range(g_size):
      for j in range(g_size):
        diffs[i][j] = ((m[i][j] - xi[IN]) ** 2).sum()
    tc = np.argmin(diffs)
    tc_i = tc // g_size
    tc_j = tc % g_size
    #
    #   勝者ベクトルの位置にプロット
    #
    plt.text(tc_i, tc_j, case_labels[IN])

  plt.show()

In [ ]:
# SOMのプログラム例（MainSom.py）

import pandas as pd
import numpy as np
import scipy.stats as ss
# from drawmap import *

#
#   Excelファイルの読み込み
#
flnm = input('Data File Name (*.xlsx) = ')
xlsx = pd.ExcelFile(flnm)      
data = pd.read_excel(xlsx) 


X = data.values
var_names = data.columns.values
print(var_names)
n = len(X[0]) - 1
JM = len(X)
print('n = ', n)
print('JM = ', JM)

case_labels = X[:,0]
print('case_labels = ', case_labels)

xi = np.empty((JM, n))  # 入力データ
for i in range(JM):
  for j in range(n):
    xi[i][j] = X[i][j + 1]

print('xi =\n', xi)

g_size = 10
m = np.empty((g_size, g_size, n))   # 参照ベクトル

for i in range(g_size):
  for j in range(g_size):
    for k in range(n):
      m[i][j][k] = ss.uniform.rvs()

t_max = 100
diffs = np.empty((g_size, g_size))
a_0 = 0.5
sgm_t = g_size / 2
for t in range(t_max):
  print('t = ', t)
  a_t = a_0 * (t_max - t) / t_max
  for IN in range(JM):
    for i in range(g_size):
      for j in range(g_size):
        diffs[i][j] = ((m[i][j] - xi[IN]) ** 2).sum()
    tc = np.argmin(diffs)   # 勝者ベクトル
    tc_i = tc // g_size    # 勝者ベクトルの行位置
    tc_j = tc % g_size    # 勝者ベクトルの列位置
    #
    #   参照ベクトルの更新
    #
    for i in range(g_size):
      for j in range(g_size):
        h_cr_IN = a_t * np.exp(-((i - tc_i)**2 + (j - tc_j)**2) / (2 * (sgm_t**2)))
        m[i][j] = m[i][j] + h_cr_IN * (xi[IN] - m[i][j])
  sgm_t = 1 + (sgm_t - 1) * (t_max - t) / t_max

#
#   マップの描画
#
draw_som(case_labels, xi, m)

In [7]:
# 描画モジュールmap2clusters.py

import matplotlib.pyplot as plt
import numpy as np

def draw_som_2clusters(case_labels, xi, m):
  JM = len(xi)
  n = len(xi[0])
  g_size = len(m)

  diffs0 = []
  diffs1 = []
  for i in range(JM):
    v = ((m[0] - xi[i]) ** 2).sum()
    diffs0.append(v)
    v = ((m[1] - xi[i]) ** 2).sum()
    diffs1.append(v)

  plt.plot(diffs0, diffs1, 'bo')
  for j in range(JM):
    plt.text(diffs0[j], diffs1[j], case_labels[j])
  plt.xlabel('$Dist.^2$ from Unit-0', fontsize = 12)
  plt.ylabel('$Dist.^2$ from Unit-1', fontsize = 12)
  plt.show()

In [ ]:
# 2個のニューロンに対するSOMを求めるプログラム（MainTwoClusters.py）

import pandas as pd
import numpy as np
import scipy.stats as ss
# from map2clusters import *

#
#   Excelファイルの読み込み
#
flnm = input('Data File Name (*.xlsx) = ')
xlsx = pd.ExcelFile(flnm)
data = pd.read_excel(xlsx)

X = data.values
var_names = data.columns.values
print(var_names)

n = len(X[0]) - 1
JM = len(X)
print('n = ', n)
print('JM = ', JM)

case_labels = X[:,0]
print('case_labels = ', case_labels)

xi = np.empty((JM, n))    # 入力データ
for i in range(JM):
  for j in range(n):
    xi[i][j] = X[i][j + 1]
print('xi =\n', xi)

g_size = 2
m = np.empty((g_size, n))   # 参照ベクトル
for i in range(g_size):
  for k in range(n):
    m[i][k] = ss.uniform.rvs()
# print('m[0] = n', m[0])

t_max = 100
diffs = np.empty(g_size)
a_0 = 0.5
sgm_t = g_size / 2
for t in range(t_max):
  print('t = ', t)
  a_t = a_0 * (t_max - t) / t_max
  for IN in range(JM):
    for i in range(g_size):
      diffs[i] = ((m[i] - xi[IN]) ** 2).sum()
    tc = np.argmin(diffs)
    for i in range(g_size):
      h_cr_IN = a_t * np.exp(-((i - tc)**2) / (2 * (sgm_t**2)))
      m[i] = m[i] + h_cr_IN * (xi[IN] - m[i])

  sgm_t = 1 + (sgm_t - 1) * (t_max - t) / t_max

draw_som_2clusters(case_labels, xi, m)